<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Bootcamp%20Qualification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#train = pd.read_csv("/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv")
#test = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')
#submission = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/submission.csv')
train = pd.read_csv("../input/dsn-ai-bootcamp-data/Train.csv")
test = pd.read_csv('../input/dsn-ai-bootcamp-data/Test.csv')
submission = pd.read_csv('../input/dsn-ai-bootcamp-data/SampleSubmission.csv')

In [ ]:
#train.info()

In [ ]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,...,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,...,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,...,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,...,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,...,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552e+07,1.326101e+07,...,167.754000,137.496750,0.628573,10.100000,0.753840,0.000000,0.000000,1.153022e+03,1.181754,4.993370e-01
max,3900.000000,18.015050,57.371600,91.672200,407.748600,5.313546e+07,2.158794e+09,1.037397e+08,3.200533e+08,2.191361e+09,...,645.450000,255.000000,2.200000,91.910000,0.800000,5.000000,5.000000,5.321014e+07,28.000000,2.307571e+08


In [ ]:
#train.isnull().sum()/len(train) * 100

In [ ]:
new_train = train.drop(columns=['form_field31','form_field40','form_field41'])
#new_train = train
#print(new_train.head())

In [ ]:
#test.isnull().sum()/len(test) * 100

In [ ]:
new_test = test.drop(columns=['form_field31','form_field40','form_field41'])
#new_test = test

# Training Catboost Algorithm with less preprocessing and feature engineering

## Feature Engineering

In [ ]:
new_train.fillna(-999, inplace=True)

In [ ]:
new_test.fillna(-999, inplace=True)

## Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
num_cols = new_train.select_dtypes(include=np.number).columns

In [ ]:
#for col in num_cols:
 # new_train[col] = np.sqrt(new_train[col])
  #new_test[col] = np.sqrt(new_test[col])

In [ ]:
pip install --quiet catboost

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
X = new_train.drop(['default_status','Applicant_ID'], axis=1)
y =new_train.default_status

In [ ]:
le = LabelEncoder()

In [ ]:
X.form_field47.value_counts()

charge     36373
lending    19627
Name: form_field47, dtype: int64

In [ ]:
X.form_field47 = le.fit_transform(X.form_field47)
new_test.form_field47 = le.fit_transform(new_test.form_field47)


In [ ]:
#X_ = (X)
test_ = (new_test.drop('Applicant_ID', axis=1))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()

In [ ]:
#X = min_max.fit_transform(X)
#test_ = min_max.fit_transform(test_)

In [ ]:
X.shape, y.shape, test_.shape

((56000, 47), (56000,), (24000, 47))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size= 0.2, random_state=42)

## Prediction

In [ ]:
from catboost import CatBoostClassifier


In [ ]:
#cat.fit(X_train, y_train , verbose=0)

In [ ]:
y_pred = cat.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

In [ ]:
pred = cat.predict_proba(test_)
submission['default_status']= pred
submission.to_csv('sub12.csv', index=False)

### CatBoost Classifier with 0.008 LR and 4000 n_estimator

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
cat = CatBoostClassifier(task_type='GPU', n_estimators=4000, learning_rate=0.008)

In [ ]:
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = cat.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

0:	learn: 0.6878907	total: 60.3ms	remaining: 4m 1s
1:	learn: 0.6825736	total: 119ms	remaining: 3m 56s
2:	learn: 0.6774316	total: 179ms	remaining: 3m 58s
3:	learn: 0.6725117	total: 233ms	remaining: 3m 53s
4:	learn: 0.6679152	total: 289ms	remaining: 3m 50s
5:	learn: 0.6631261	total: 345ms	remaining: 3m 49s
6:	learn: 0.6582802	total: 399ms	remaining: 3m 47s
7:	learn: 0.6536723	total: 452ms	remaining: 3m 45s
8:	learn: 0.6490751	total: 507ms	remaining: 3m 44s
9:	learn: 0.6449173	total: 561ms	remaining: 3m 43s
10:	learn: 0.6407733	total: 615ms	remaining: 3m 42s
11:	learn: 0.6366818	total: 668ms	remaining: 3m 42s
12:	learn: 0.6325934	total: 725ms	remaining: 3m 42s
13:	learn: 0.6285034	total: 778ms	remaining: 3m 41s
14:	learn: 0.6246652	total: 832ms	remaining: 3m 41s
15:	learn: 0.6209838	total: 886ms	remaining: 3m 40s
16:	learn: 0.6175534	total: 925ms	remaining: 3m 36s
17:	learn: 0.6141098	total: 978ms	remaining: 3m 36s
18:	learn: 0.6103922	total: 1.03s	remaining: 3m 36s
19:	learn: 0.6071803	t

160:	learn: 0.4401450	total: 9.57s	remaining: 3m 48s
161:	learn: 0.4398514	total: 9.62s	remaining: 3m 47s
162:	learn: 0.4395495	total: 9.67s	remaining: 3m 47s
163:	learn: 0.4392761	total: 9.73s	remaining: 3m 47s
164:	learn: 0.4389529	total: 9.78s	remaining: 3m 47s
165:	learn: 0.4386899	total: 9.84s	remaining: 3m 47s
166:	learn: 0.4383742	total: 9.89s	remaining: 3m 47s
167:	learn: 0.4380658	total: 9.94s	remaining: 3m 46s
168:	learn: 0.4377703	total: 10s	remaining: 3m 46s
169:	learn: 0.4374964	total: 10.1s	remaining: 3m 46s
170:	learn: 0.4372314	total: 10.1s	remaining: 3m 45s
171:	learn: 0.4369766	total: 10.1s	remaining: 3m 45s
172:	learn: 0.4366915	total: 10.2s	remaining: 3m 46s
173:	learn: 0.4364403	total: 10.3s	remaining: 3m 45s
174:	learn: 0.4361565	total: 10.3s	remaining: 3m 45s
175:	learn: 0.4359168	total: 10.4s	remaining: 3m 45s
176:	learn: 0.4357171	total: 10.4s	remaining: 3m 45s
177:	learn: 0.4354248	total: 10.5s	remaining: 3m 45s
178:	learn: 0.4351598	total: 10.5s	remaining: 3m

316:	learn: 0.4171269	total: 18.6s	remaining: 3m 35s
317:	learn: 0.4170701	total: 18.6s	remaining: 3m 35s
318:	learn: 0.4170006	total: 18.6s	remaining: 3m 35s
319:	learn: 0.4169449	total: 18.7s	remaining: 3m 35s
320:	learn: 0.4168799	total: 18.8s	remaining: 3m 34s
321:	learn: 0.4168159	total: 18.8s	remaining: 3m 34s
322:	learn: 0.4167668	total: 18.9s	remaining: 3m 34s
323:	learn: 0.4166914	total: 18.9s	remaining: 3m 34s
324:	learn: 0.4166272	total: 19s	remaining: 3m 34s
325:	learn: 0.4165665	total: 19s	remaining: 3m 34s
326:	learn: 0.4164970	total: 19.1s	remaining: 3m 34s
327:	learn: 0.4164403	total: 19.1s	remaining: 3m 34s
328:	learn: 0.4163763	total: 19.2s	remaining: 3m 34s
329:	learn: 0.4163213	total: 19.3s	remaining: 3m 34s
330:	learn: 0.4162549	total: 19.3s	remaining: 3m 34s
331:	learn: 0.4161810	total: 19.4s	remaining: 3m 33s
332:	learn: 0.4161073	total: 19.4s	remaining: 3m 33s
333:	learn: 0.4160438	total: 19.5s	remaining: 3m 33s
334:	learn: 0.4159713	total: 19.5s	remaining: 3m 3

473:	learn: 0.4100065	total: 27s	remaining: 3m 20s
474:	learn: 0.4099860	total: 27.1s	remaining: 3m 20s
475:	learn: 0.4099537	total: 27.1s	remaining: 3m 20s
476:	learn: 0.4099237	total: 27.2s	remaining: 3m 20s
477:	learn: 0.4099040	total: 27.2s	remaining: 3m 20s
478:	learn: 0.4098714	total: 27.3s	remaining: 3m 20s
479:	learn: 0.4098441	total: 27.3s	remaining: 3m 20s
480:	learn: 0.4098130	total: 27.4s	remaining: 3m 20s
481:	learn: 0.4097799	total: 27.4s	remaining: 3m 20s
482:	learn: 0.4097494	total: 27.5s	remaining: 3m 20s
483:	learn: 0.4097236	total: 27.5s	remaining: 3m 20s
484:	learn: 0.4096805	total: 27.6s	remaining: 3m 20s
485:	learn: 0.4096545	total: 27.6s	remaining: 3m 19s
486:	learn: 0.4096322	total: 27.7s	remaining: 3m 19s
487:	learn: 0.4096081	total: 27.8s	remaining: 3m 19s
488:	learn: 0.4095739	total: 27.8s	remaining: 3m 19s
489:	learn: 0.4095367	total: 27.9s	remaining: 3m 19s
490:	learn: 0.4095140	total: 27.9s	remaining: 3m 19s
491:	learn: 0.4094983	total: 28s	remaining: 3m 1

632:	learn: 0.4063506	total: 36.1s	remaining: 3m 12s
633:	learn: 0.4063266	total: 36.2s	remaining: 3m 12s
634:	learn: 0.4063045	total: 36.2s	remaining: 3m 12s
635:	learn: 0.4062891	total: 36.3s	remaining: 3m 11s
636:	learn: 0.4062690	total: 36.3s	remaining: 3m 11s
637:	learn: 0.4062541	total: 36.4s	remaining: 3m 11s
638:	learn: 0.4062380	total: 36.5s	remaining: 3m 11s
639:	learn: 0.4062085	total: 36.5s	remaining: 3m 11s
640:	learn: 0.4062011	total: 36.6s	remaining: 3m 11s
641:	learn: 0.4061789	total: 36.6s	remaining: 3m 11s
642:	learn: 0.4061444	total: 36.7s	remaining: 3m 11s
643:	learn: 0.4061332	total: 36.7s	remaining: 3m 11s
644:	learn: 0.4061222	total: 36.8s	remaining: 3m 11s
645:	learn: 0.4061072	total: 36.8s	remaining: 3m 11s
646:	learn: 0.4060795	total: 36.9s	remaining: 3m 11s
647:	learn: 0.4060453	total: 36.9s	remaining: 3m 11s
648:	learn: 0.4060257	total: 37s	remaining: 3m 10s
649:	learn: 0.4060024	total: 37s	remaining: 3m 10s
650:	learn: 0.4059831	total: 37.1s	remaining: 3m 1

...

3318:	learn: 0.3901791	total: 3m 8s	remaining: 38.6s
3319:	learn: 0.3901724	total: 3m 8s	remaining: 38.6s
3320:	learn: 0.3901669	total: 3m 8s	remaining: 38.5s
3321:	learn: 0.3901547	total: 3m 8s	remaining: 38.4s
3322:	learn: 0.3901452	total: 3m 8s	remaining: 38.4s
3323:	learn: 0.3901408	total: 3m 8s	remaining: 38.3s
3324:	learn: 0.3901314	total: 3m 8s	remaining: 38.3s
3325:	learn: 0.3901266	total: 3m 8s	remaining: 38.2s
3326:	learn: 0.3901253	total: 3m 8s	remaining: 38.2s
3327:	learn: 0.3901184	total: 3m 8s	remaining: 38.1s
3328:	learn: 0.3901168	total: 3m 8s	remaining: 38s
3329:	learn: 0.3901161	total: 3m 8s	remaining: 38s
3330:	learn: 0.3901157	total: 3m 8s	remaining: 37.9s
3331:	learn: 0.3901140	total: 3m 8s	remaining: 37.9s
3332:	learn: 0.3901134	total: 3m 8s	remaining: 37.8s
3333:	learn: 0.3901020	total: 3m 8s	remaining: 37.8s
3334:	learn: 0.3900938	total: 3m 9s	remaining: 37.7s
3335:	learn: 0.3900871	total: 3m 9s	remaining: 37.6s
3336:	learn: 0.3900864	total: 3m 9s	remaining: 37.

3472:	learn: 0.3893380	total: 3m 16s	remaining: 29.9s
3473:	learn: 0.3893371	total: 3m 17s	remaining: 29.8s
3474:	learn: 0.3893272	total: 3m 17s	remaining: 29.8s
3475:	learn: 0.3893216	total: 3m 17s	remaining: 29.7s
3476:	learn: 0.3893172	total: 3m 17s	remaining: 29.7s
3477:	learn: 0.3893168	total: 3m 17s	remaining: 29.6s
3478:	learn: 0.3893061	total: 3m 17s	remaining: 29.5s
3479:	learn: 0.3892992	total: 3m 17s	remaining: 29.5s
3480:	learn: 0.3892989	total: 3m 17s	remaining: 29.4s
3481:	learn: 0.3892951	total: 3m 17s	remaining: 29.4s
3482:	learn: 0.3892869	total: 3m 17s	remaining: 29.3s
3483:	learn: 0.3892862	total: 3m 17s	remaining: 29.3s
3484:	learn: 0.3892742	total: 3m 17s	remaining: 29.2s
3485:	learn: 0.3892671	total: 3m 17s	remaining: 29.1s
3486:	learn: 0.3892645	total: 3m 17s	remaining: 29.1s
3487:	learn: 0.3892587	total: 3m 17s	remaining: 29s
3488:	learn: 0.3892493	total: 3m 17s	remaining: 29s
3489:	learn: 0.3892395	total: 3m 17s	remaining: 28.9s
3490:	learn: 0.3892306	total: 3m

3627:	learn: 0.3884164	total: 3m 25s	remaining: 21.1s
3628:	learn: 0.3884132	total: 3m 25s	remaining: 21s
3629:	learn: 0.3884131	total: 3m 25s	remaining: 21s
3630:	learn: 0.3884115	total: 3m 25s	remaining: 20.9s
3631:	learn: 0.3884102	total: 3m 25s	remaining: 20.9s
3632:	learn: 0.3884085	total: 3m 25s	remaining: 20.8s
3633:	learn: 0.3883994	total: 3m 26s	remaining: 20.7s
3634:	learn: 0.3883990	total: 3m 26s	remaining: 20.7s
3635:	learn: 0.3883904	total: 3m 26s	remaining: 20.6s
3636:	learn: 0.3883874	total: 3m 26s	remaining: 20.6s
3637:	learn: 0.3883813	total: 3m 26s	remaining: 20.5s
3638:	learn: 0.3883810	total: 3m 26s	remaining: 20.5s
3639:	learn: 0.3883725	total: 3m 26s	remaining: 20.4s
3640:	learn: 0.3883711	total: 3m 26s	remaining: 20.4s
3641:	learn: 0.3883696	total: 3m 26s	remaining: 20.3s
3642:	learn: 0.3883690	total: 3m 26s	remaining: 20.2s
3643:	learn: 0.3883490	total: 3m 26s	remaining: 20.2s
3644:	learn: 0.3883400	total: 3m 26s	remaining: 20.1s
3645:	learn: 0.3883388	total: 3m

3783:	learn: 0.3875962	total: 3m 34s	remaining: 12.2s
3784:	learn: 0.3875958	total: 3m 34s	remaining: 12.2s
3785:	learn: 0.3875902	total: 3m 34s	remaining: 12.1s
3786:	learn: 0.3875817	total: 3m 34s	remaining: 12.1s
3787:	learn: 0.3875780	total: 3m 34s	remaining: 12s
3788:	learn: 0.3875778	total: 3m 34s	remaining: 12s
3789:	learn: 0.3875672	total: 3m 34s	remaining: 11.9s
3790:	learn: 0.3875669	total: 3m 34s	remaining: 11.8s
3791:	learn: 0.3875608	total: 3m 34s	remaining: 11.8s
3792:	learn: 0.3875572	total: 3m 34s	remaining: 11.7s
3793:	learn: 0.3875487	total: 3m 34s	remaining: 11.7s
3794:	learn: 0.3875476	total: 3m 35s	remaining: 11.6s
3795:	learn: 0.3875381	total: 3m 35s	remaining: 11.6s
3796:	learn: 0.3875349	total: 3m 35s	remaining: 11.5s
3797:	learn: 0.3875339	total: 3m 35s	remaining: 11.4s
3798:	learn: 0.3875255	total: 3m 35s	remaining: 11.4s
3799:	learn: 0.3875165	total: 3m 35s	remaining: 11.3s
3800:	learn: 0.3875134	total: 3m 35s	remaining: 11.3s
3801:	learn: 0.3875061	total: 3m

3938:	learn: 0.3868207	total: 3m 42s	remaining: 3.45s
3939:	learn: 0.3868202	total: 3m 42s	remaining: 3.39s
3940:	learn: 0.3868202	total: 3m 42s	remaining: 3.34s
3941:	learn: 0.3868199	total: 3m 42s	remaining: 3.28s
3942:	learn: 0.3868197	total: 3m 43s	remaining: 3.22s
3943:	learn: 0.3868192	total: 3m 43s	remaining: 3.17s
3944:	learn: 0.3868045	total: 3m 43s	remaining: 3.11s
3945:	learn: 0.3868042	total: 3m 43s	remaining: 3.06s
3946:	learn: 0.3867953	total: 3m 43s	remaining: 3s
3947:	learn: 0.3867929	total: 3m 43s	remaining: 2.95s
3948:	learn: 0.3867909	total: 3m 43s	remaining: 2.89s
3949:	learn: 0.3867814	total: 3m 43s	remaining: 2.83s
3950:	learn: 0.3867770	total: 3m 43s	remaining: 2.78s
3951:	learn: 0.3867769	total: 3m 44s	remaining: 2.72s
3952:	learn: 0.3867677	total: 3m 44s	remaining: 2.66s
3953:	learn: 0.3867538	total: 3m 44s	remaining: 2.61s
3954:	learn: 0.3867534	total: 3m 44s	remaining: 2.55s
3955:	learn: 0.3867457	total: 3m 44s	remaining: 2.49s
3956:	learn: 0.3867392	total: 3

In [ ]:
np.mean(roc_score)

0.8394334827495037

In [ ]:
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('kaggle_cat1.csv', index=False)

In [ ]:
np.mean(roc_score)

0.838164390501938

In [ ]:
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('kaggle_cat.csv', index=False)

## GridSearch 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [ ]:
grid = {'n_estimators': [ 1000] }
#'depth': [4,6]}
       
        
        #'l2_leaf_reg': [3, 5]}
        #{'learning_rate': [ 0.02, 0.03],
cat = CatBoostClassifier(task_type='GPU', learning_rate=0.02,  verbose=0, early_stopping_rounds=100)

In [ ]:
rscv = RandomizedSearchCV(cat, grid )

### Randomized Search with Train_test_split

In [ ]:
model =rscv.fit(X_train, y_train)

In [ ]:
rscv.best_estimator_, model.best_params_


In [ ]:
y_pred = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('prediction_results04.csv')         


In [ ]:
from google.colab import files
pred = model.predict_proba(test_)[:,1]
submission.Applicant_ID = test.Applicant_ID
submission['default_status']= pred
submission.to_csv('prediction_results05.csv', lindex=Fase)         
files.download('prediction_results05.csv')

In [ ]:
y_pred = rscv.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

In [ ]:
rscv.best_estimator_, rscv.best_params_

In [ ]:
pred = rscv.predict_proba(test_)[:,1]
submission['default_status']= pred
submission.to_csv('sub17.csv', index=False)

## Using Parameters from Randomized SearchCV with KFold

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
cat = CatBoostClassifier(task_type='GPU', learning_rate=0.02, n_estimators=1000)


In [ ]:
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = cat.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
np.mean(roc_score)

In [ ]:
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('sub18.csv', index=False)

In [ ]:
#from google.colab import files
res.to_csv('prediction_results.csv')         
#files.download('prediction_results.csv')

## Trying ensemble techniques

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
xgb = XGBClassifier()
lgb = LGBMClassifier()
ada = AdaBoostClassifier()
rf = RandomForestClassifier()

gb = GradientBoostingClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

### USing XGBoost for prediction

In [ ]:
kf = KFold(n_splits=5)
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = xgb.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
np.mean(roc_score)

In [ ]:
from google.colab import files
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('xgb.csv', index=False)
files.download('xgb.csv')


### Prediction with LGBMClassifier

In [ ]:
kf = KFold(n_splits=5)
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = lgb.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
np.mean(roc_score)


In [ ]:
from google.colab import files
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('lgb.csv', index=False)
files.download('lgb.csv')

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

### Voting Classifier

In [ ]:
cat = CatBoostClassifier(task_type='GPU', verbose=0)

In [ ]:
# Defning the ensemble model using VotingClassifer
voting = VotingClassifier(estimators=[('xgb', xgb),('lgb', lgb),('cat',cat)],
voting= 'soft')

In [ ]:
kf = KFold(n_splits=5)
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = voting.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
np.mean(roc_score)

In [ ]:
from google.colab import files
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('voting.csv', index=False)
files.download('voting.csv')

### Prediction with StackingClassifier

In [ ]:
sc = StackingClassifier([('lgb',lgb), ('xgb',xgb), ('cat', cat)], final_estimator=lr)

In [ ]:
kf = KFold(n_splits=5)
#X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X,y):
    x_train, x_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = sc.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
np.mean(roc_score)

In [ ]:
from google.colab import files
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('stacking.csv', index=False)
files.download('stacking.csv')

In [ ]:
"""from google.colab import files
pred = model.predict_proba(test_)[:,1]
submission.Applicant_ID = test.Applicant_ID
submission['default_status']= pred
submission.to_csv('prediction_results05.csv', index=False)         
files.download('prediction_results05.csv')"""

### Kfold CV with 3 and 5 splits

3 splits showed improvement in roc score over 5 splits, but 5 splits performed better on LB

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
X_= pd.DataFrame(X)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X_,y):
    x_train, x_test = X_.iloc[train_index],X_.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = cat.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)

In [ ]:
#prediction = gscv.predict_proba(test_sc)[:,1]
np.mean(roc_score)


In [ ]:
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('sub16.csv', index=False)

### Binning

In [ ]:
num_cols = new_train.select_dtypes(exclude='object').columns

In [ ]:
#for num in num_cols:
 # new_train[num + '_bin'] = pd.cut(new_train[num],bins=4)# duplicates='drop' 
 # new_test[num + '_bin'] = pd.cut(new_test[num],bins=4 )#duplicates='drop' 

#new_train['form_field1_bin'] = pd.cut(new_train.form_field1, bins=4)
#sns.countplot(s)

In [ ]:
#sns.countplot(new_train.form_field18_bin)

### Feature Interaction

In [ ]:
new_train['form_field1*formfield2'] = new_train['form_field1'] * new_train['form_field2']
new_train['form_field1/formfield2'] = new_train['form_field1'] / new_train['form_field2']

new_train['form_field1*formfield3'] = new_train['form_field1'] * new_train.form_field3
new_train['form_field1*formfield4'] = new_train.form_field1 * new_train.form_field4
new_train['form_field1*formfield5'] = new_train.form_field1 * new_train.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

new_train['form_field6+field7'] =  (new_train.form_field6 + new_train.form_field8)/2
new_train['form_field13/field14'] = (new_train.form_field13 / new_train.form_field14)*100
new_train['form_field32/field33'] = (new_train['form_field32']+ new_train['form_field33'])/2
new_train['form_field36/field38'] = new_train['form_field36'] / new_train['form_field38']
new_train['form_field36/field39'] = new_train['form_field36'] / new_train['form_field39']
new_train['form_field36/field18'] = new_train['form_field36'] / new_train['form_field18']
new_train['form_field36/field20'] = new_train['form_field36'] / new_train['form_field20']
new_train['form_field44*field50'] = new_train['form_field44'] * new_train['form_field50']




In [ ]:
new_test['form_field1*formfield2'] = new_test['form_field1'] * new_test['form_field2']
new_test['form_field1/formfield2'] = new_test['form_field1'] / new_test['form_field2']

new_test['form_field1*formfield3'] = new_test['form_field1'] * new_test.form_field3
new_test['form_field1*formfield4'] = new_test.form_field1 * new_test.form_field4
new_test['form_field1*formfield5'] = new_test.form_field1 * new_test.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

new_test['form_field6+field7'] =  (new_test.form_field6 + new_test.form_field8)/2
new_test['form_field13/field14'] = (new_test.form_field13 / new_test.form_field14)*100
new_test['form_field32/field33'] = (new_test['form_field32']+ new_test['form_field33'])/2
new_test['form_field36/field38'] = new_test['form_field36'] / new_test['form_field38']
new_test['form_field36/field39'] = new_test['form_field36'] / new_test['form_field39']
new_test['form_field36/field18'] = new_test['form_field36'] / new_test['form_field18']
new_test['form_field36/field20'] = new_test['form_field36'] / new_test['form_field20']
new_test['form_field44*field50'] = new_test['form_field44'] * new_test['form_field50']




### Data Aggregation

In [ ]:
for col in num_cols:
    new_train[col + '_mean_on_field1'] = new_train[col].groupby(new_train['form_field1']).transform('mean')
    new_train[col + '_mean_on_field2'] = new_train[col].groupby(new_train['form_field2']).transform('mean')

    new_train[col + '_sum_on_field1'] = new_train[col].groupby(new_train['form_field1']).transform('sum')
    new_train[col + '_sum_on_field2'] = new_train[col].groupby(new_train['form_field2']).transform('sum')

    new_train[col + '_mean_on_field1'] = new_train[col].groupby(new_train['form_field1']).transform('max')
    new_train[col + '_mean_on_field2'] = new_train[col].groupby(new_train['form_field2']).transform('max')

    new_train[col + '_sum_on_field1'] = new_train[col].groupby(new_train['form_field1']).transform('min')
    new_train[col + '_sum_on_field2'] = new_train[col].groupby(new_train['form_field2']).transform('min')


    new_test[col + '_mean_on_field1'] = new_test[col].groupby(new_test['form_field1']).transform('mean')
    new_test[col + '_mean_on_field2'] = new_test[col].groupby(new_test['form_field2']).transform('mean')

    new_test[col + '_sum_on_field1'] = new_test[col].groupby(new_test['form_field1']).transform('sum')
    new_test[col + '_sum_on_field2'] = new_test[col].groupby(new_test['form_field2']).transform('sum')

    new_test[col + '_mean_on_field1'] = new_test[col].groupby(new_test['form_field1']).transform('max')
    new_test[col + '_mean_on_field2'] = new_test[col].groupby(new_test['form_field2']).transform('max')

    new_test[col + '_sum_on_field1'] = new_test[col].groupby(new_test['form_field1']).transform('min')
    new_test[col + '_sum_on_field2'] = new_test[col].groupby(new_test['form_field2']).transform('min')




    new_train[col + '_mean_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('mean')
    new_train[col + '_mean_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('mean')

    new_train[col + '_sum_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('sum')
    new_train[col + '_sum_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('sum')

    new_train[col + '_mean_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('max')
    new_train[col + '_mean_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('max')

    new_train[col + '_sum_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('min')
    new_train[col + '_sum_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('min')


    new_test[col + '_mean_on_field1_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field1']]).transform('mean')
    new_test[col + '_mean_on_field2_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field2']]).transform('mean')

    new_test[col + '_sum_on_field1_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field1']]).transform('sum')
    new_test[col + '_sum_on_field2_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field2']]).transform('sum')

    new_test[col + '_mean_on_field1_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field1']]).transform('max')
    new_test[col + '_mean_on_field2_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field2']]).transform('max')

    new_test[col + '_sum_on_field1_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field1']]).transform('min')
    new_test[col + '_sum_on_field2_'] = new_test[col].groupby([new_train['form_field47'],new_test['form_field2']]).transform('min')
    
    
    

In [ ]:
#new_train[col].groupby([new_train['form_field1'],new_train['form_field47']]).transform('mean')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
X = new_train.drop(['default_status','Applicant_ID'], axis=1)
y =new_train.default_status

In [ ]:
#y_ = pd.get_dummies(y)

In [ ]:
X.form_field47 = le.fit_transform(X.form_field47)
new_test.form_field47 = le.fit_transform(new_test.form_field47)


In [ ]:
X_ = (X)
test_ = (new_test.drop('Applicant_ID', axis=1))

In [ ]:
#cat_cols = X_.select_dtypes(include='category').columns
#cat_cols

In [ ]:
test_.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
minmax = MinMaxScaler()

In [ ]:
#X_scaled = minmax.fit_transform(X_)
#test_sc = minmax.fit_transform(test_)

In [ ]:
pip install --quiet catboost

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier



In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
cat = CatBoostClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
ada = AdaBoostClassifier()
rf = RandomForestClassifier()
lr = LogisticRegression()

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV, cross_val_score

In [ ]:
from sklearn.metrics import roc_auc_score

## Applied Kfold CV

In [ ]:
kf = KFold(n_splits=5)
cat = CatBoostClassifier(task_type='GPU')#, cat_features=cat_cols)#, learning_rate=0.02, n_estimators=2000, depth=7, early_stopping_rounds=50)


In [ ]:
X_scaled1 = pd.DataFrame(X_)

roc_score = []
test_pred = []

for train_index, test_index in kf.split(X_scaled1,y):
    x_train, x_test = X_scaled1.iloc[train_index],X_scaled1.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    
    model = cat.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)[:,1]
    score = roc_auc_score(y_test,y_pred)
    pred = model.predict_proba(test_)[:,1]
    roc_score.append(score)
    test_pred.append(pred)
  #save result to list

In [ ]:
np.mean(roc_score)

In [ ]:
test

In [ ]:
submission['Applicant_ID'] = test['Applicant_ID']

In [ ]:
#prediction = gscv.predict_proba(test_sc)[:,1]
np.mean(roc_score)
submission['default_status']= np.mean(test_pred, axis=0)
submission.to_csv('sub10.csv', index=False)

In [ ]:
cat.feature_importances_

In [ ]:
feature_importance_df = pd.DataFrame(cat.feature_importances_, columns=['importance'])
feature_importance_df['feature'] = test_.columns

plt.figure(figsize=(12, 6));
sns.barplot(x="importance", y="feature", data=feature_importance_df.sort_values(by = ['importance'], ascending = False).head(30))
plt.title('RandomForestClassifier features importance:');

In [ ]:
grid = {'learning_rate': [ 0.05],}
        #'n_estimators': [300, 500] }
       # 'depth': [4,6],}
        #'l2_leaf_reg': [3, 5]}

"""grid_search_result = cat.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       plot=True)"""

In [ ]:
cat = CatBoostClassifier(task_type='GPU', depth=7, n_estimators=1500, early_stopping_rounds=50)


gscv = GridSearchCV(cat, param_grid=grid)

Tried GridSearch CV

In [ ]:
gscv.fit(X_train, y_train)

In [ ]:
gscv_pred = gscv.predict_proba(X_test)[:,1]
roc_auc_score(y_test, gscv_pred)


In [ ]:
\\prediction = gscv.predict_proba(test_sc)[:,1]

submission['default_status']=prediction
submission.to_csv('submission4.csv', index=False)


In [ ]:
cat = CatBoostClassifier(task_type='GPU', learning_rate=0.03, depth=4, n_estimators=3000, early_stopping_rounds=50)

cat.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss

In [ ]:
pred = cat.predict_proba(X_test)[:,1]

In [ ]:
roc_auc_score(y_test, pred)


In [ ]:
roc_auc_score(y_test, pred)


In [ ]:
roc_auc_score(y_test, pred)

In [ ]:
models = [cat, xgb, lgb, ada, rf, lr]

for model in models:
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test,pred)
    print(f'the auc score for {model} is {score}')

In [ ]:
prediction = cat.predict_proba(test_sc)[:,1]

In [ ]:
submission.head()

In [ ]:
prediction = cat.predict_proba(test_sc)[:,1]

submission['default_status']=prediction
submission.to_csv('submission3.csv', index=False)


In [ ]:
submission.to_csv('submission3.csv', index=False)

In [ ]:
len(cat.feature_importances_)
len(test_.columns)

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
feature_importance_df = pd.DataFrame(cat.feature_importances_, columns=['importance'])
feature_importance_df['feature'] = test_.columns

plt.figure(figsize=(12, 6));
sns.barplot(x="importance", y="feature", data=feature_importance_df.sort_values(by = ['importance'], ascending = False).head(30))
plt.title('RandomForestClassifier features importance:');

## Feature Engineering

In [ ]:
new_train.loc[new_train.form_field1>=3500,'form_field1']=1